# **02 - Preprocessing**

In [1]:
import sys
sys.path.insert(0, "../src")

import pandas as pd
import numpy as np
from dataset import Dataset

## **Verify normalization across all years**

Each year must produce the exact same column set regardless of its format group.

In [2]:
from dataset import (
    CANONICAL_CHARACTERISTICS,
    CANONICAL_LOCATIONS,
    CANONICAL_VEHICLES,
    CANONICAL_USERS,
)

for y in list(range(2010, 2023)) + ["test"]:
    ds = Dataset(y)
    assert ds.characteristics.columns.tolist() == CANONICAL_CHARACTERISTICS, f"{y}: characteristics columns mismatch"
    assert ds.locations.columns.tolist() == CANONICAL_LOCATIONS, f"{y}: locations columns mismatch"
    assert ds.vehicles.columns.tolist() == CANONICAL_VEHICLES, f"{y}: vehicles columns mismatch"
    assert ds.users.columns.tolist() == CANONICAL_USERS, f"{y}: users columns mismatch"
    n_acc = ds.characteristics["Num_Acc"].nunique()
    print(f"{str(y):5s} (group {ds.group}) : {n_acc:>6,} accidents  OK")

print("\nAll years normalized successfully.")

2010  (group A) : 69,379 accidents  OK
2011  (group A) : 66,974 accidents  OK
2012  (group B) : 56,025 accidents  OK
2013  (group B) : 52,558 accidents  OK
2014  (group B) : 53,869 accidents  OK
2015  (group B) : 52,789 accidents  OK
2016  (group B) : 53,489 accidents  OK
2017  (group B) : 54,631 accidents  OK
2018  (group B) : 52,005 accidents  OK
2019  (group C) : 52,956 accidents  OK
2020  (group C) : 42,970 accidents  OK
2021  (group C) : 50,867 accidents  OK
2022  (group C) : 49,772 accidents  OK
test  (group D) : 63,544 accidents  OK

All years normalized successfully.


## **Check for unexpected values**

For each categorical column, verify that every value belongs to the official BAAC codebook. Any value outside the expected set is flagged.

In [3]:
found_issues = False

for y in list(range(2010, 2023)) + ["test"]:
    ds = Dataset(y)
    issues = ds.check_values()
    for table, unexpected in issues.items():
        if unexpected:
            found_issues = True
            for col, vals in unexpected.items():
                print(f"{str(y):5s} | {table:20s} | {col:8s} → unexpected: {vals}")

if not found_issues:
    print("All values are within expected ranges.")

2010  | users                | catu     → unexpected: {np.int8(4)}
2011  | users                | catu     → unexpected: {np.int8(4)}
2012  | users                | catu     → unexpected: {np.int8(4)}
2013  | users                | catu     → unexpected: {np.int8(4)}
2014  | users                | catu     → unexpected: {np.int8(4)}
2015  | users                | catu     → unexpected: {np.int8(4)}
2016  | users                | catu     → unexpected: {np.int8(4)}
2017  | users                | catu     → unexpected: {np.int8(4)}
test  | users                | catu     → unexpected: {np.int8(4)}


In [4]:
# Aggregate across all years + test
all_users = []
all_merged = []
for y in list(range(2010, 2023)) + ["test"]:
    ds = Dataset(y)
    all_users.append(ds.users)
    all_merged.append(ds.merged())
all_users = pd.concat(all_users, ignore_index=True)
all_merged = pd.concat(all_merged, ignore_index=True)

print("=== etatp=0 vs catu (all years) ===")
print(all_users[all_users["etatp"] == 0]["catu"].value_counts())
print(f"\nTotal etatp=0: {(all_users['etatp'] == 0).sum()}")
print(f"  of which catu=3 (pedestrian): {((all_users['etatp'] == 0) & (all_users['catu'] == 3)).sum()}")

print("\n=== catv=-1 vs catu (all years, merged) ===")
print(all_merged[all_merged["catv"] == -1]["catu"].value_counts())
print(f"\nTotal catv=-1: {(all_merged['catv'] == -1).sum()}")
print(f"  of which catu=3 (pedestrian): {((all_merged['catv'] == -1) & (all_merged['catu'] == 3)).sum()}")

=== etatp=0 vs catu (all years) ===
Series([], Name: count, dtype: Int64)

Total etatp=0: 0
  of which catu=3 (pedestrian): 0

=== catv=-1 vs catu (all years, merged) ===
Series([], Name: count, dtype: Int64)

Total catv=-1: 0
  of which catu=3 (pedestrian): 0


In [5]:
# Does etatp apply only to the pedestrian row, or to all users in the same accident?
# Find accidents that have at least one pedestrian with etatp != 0
accidents_with_ped = all_users[
    (all_users["catu"] == 3) & (all_users["etatp"].isin([1, 2, 3]))
]["Num_Acc"].unique()

subset = all_users[all_users["Num_Acc"].isin(accidents_with_ped)]

print(f"Accidents with a pedestrian having etatp in {{1,2,3}}: {len(accidents_with_ped):,}")
print(f"Total rows in those accidents: {len(subset):,}\n")

print("=== etatp distribution for PEDESTRIANS (catu=3) ===")
print(subset[subset["catu"] == 3]["etatp"].value_counts().sort_index())

print("\n=== etatp distribution for NON-PEDESTRIANS (catu!=3) ===")
print(subset[subset["catu"] != 3]["etatp"].value_counts().sort_index())

print("\n=== Sample accident ===")
sample_acc = accidents_with_ped[0]
print(all_users[all_users["Num_Acc"] == sample_acc][["Num_Acc", "catu", "etatp", "locp", "actp"]])

Accidents with a pedestrian having etatp in {1,2,3}: 131,038
Total rows in those accidents: 273,429

=== etatp distribution for PEDESTRIANS (catu=3) ===
etatp
-1       187
1     105932
2      27780
3       5917
Name: count, dtype: Int64

=== etatp distribution for NON-PEDESTRIANS (catu!=3) ===
Series([], Name: count, dtype: Int64)

=== Sample accident ===
        Num_Acc  catu  etatp  locp  actp
1  201000000002     1   <NA>  <NA>  <NA>
2  201000000002     3      1     4     3


## **NaN vs -1 audit**

Check if any column has both NaN and -1 values, which would indicate `-1` is a duplicate encoding for "unknown".

In [6]:
# For each numeric column, check if both NaN and -1 coexist
rows = []
for col in all_merged.select_dtypes(include="number").columns:
    nan_count = int(all_merged[col].isna().sum())
    minus1_count = int((all_merged[col] == -1).sum())
    if nan_count > 0 and minus1_count > 0:
        rows.append({
            "column": col,
            "NaN_count": nan_count,
            "-1_count": minus1_count,
            "total": len(all_merged),
        })

if rows:
    audit = pd.DataFrame(rows).sort_values("-1_count", ascending=False)
    audit["NaN_%"] = (audit["NaN_count"] / audit["total"] * 100).round(2)
    audit["-1_%"] = (audit["-1_count"] / audit["total"] * 100).round(2)
    print(audit[["column", "NaN_count", "NaN_%", "-1_count", "-1_%"]].to_string(index=False))
else:
    print("No column has both NaN and -1 values.")

column  NaN_count  NaN_%  -1_count  -1_%
  locp    1580423  91.55     10825  0.63
 etatp    1580403  91.55      6228  0.36
  actp    1582028  91.64      5987  0.35


## **Build full training set**

In [8]:
train = pd.concat(
    [Dataset(y).merged() for y in range(2010, 2023)],
    ignore_index=True,
)
print(f"Full training set: {train.shape[0]:,} rows x {train.shape[1]} cols")
print(f"Unique accidents:  {train['Num_Acc'].nunique():,}")
train.head()

Full training set: 1,583,848 rows x 46 cols
Unique accidents:  708,284


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,place,catu,grav,sexe,an_nais,trajet,secu1,locp,actp,etatp
0,201000000001,2010,6,12,1930,1,2,1,1,6,...,1,1,3,1,1976,5,2,<NA>,<NA>,<NA>
1,201000000002,2010,8,7,1000,1,2,1,1,6,...,1,1,1,2,1983,5,1,<NA>,<NA>,<NA>
2,201000000002,2010,8,7,1000,1,2,1,1,6,...,<NA>,3,3,1,1965,<NA>,<NA>,4,3,1
3,201000000003,2010,9,11,1600,1,1,1,1,5,...,1,1,3,1,1979,5,<NA>,<NA>,<NA>,<NA>
4,201000000003,2010,9,11,1600,1,1,1,1,5,...,1,1,1,2,1959,5,1,<NA>,<NA>,<NA>


## **Build test set**

In [9]:
test = Dataset("test").merged()
print(f"Test set: {test.shape[0]:,} rows x {test.shape[1]} cols")
print(f"Unique accidents: {test['Num_Acc'].nunique():,}")
print(f"grav column (should be all NaN): {test['grav'].isna().all()}")
test.head()

Test set: 142,422 rows x 46 cols
Unique accidents: 63,544
grav column (should be all NaN): True


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,place,catu,grav,sexe,an_nais,trajet,secu1,locp,actp,etatp
0,201200049538,2012,8,12,1320,1,1,1,1,4,...,1,1,<NA>,2,1954,1,1,<NA>,<NA>,<NA>
1,201200049538,2012,8,12,1320,1,1,1,1,4,...,1,1,<NA>,2,1968,1,1,<NA>,<NA>,<NA>
2,201200049538,2012,8,12,1320,1,1,1,1,4,...,1,1,<NA>,2,1984,2,1,<NA>,<NA>,<NA>
3,201200004221,2012,6,22,615,2,1,1,1,6,...,1,1,<NA>,1,1973,<NA>,2,<NA>,<NA>,<NA>
4,201200002457,2012,2,3,2245,3,1,1,1,1,...,1,1,<NA>,1,1984,<NA>,1,<NA>,<NA>,<NA>
